# Construction de la bdd filtrée 

In [2]:
import pandas as pd

In [3]:
enedis_final=pd.read_csv("Données/enedis_final.csv")

On ne garde que les données inférieure au 31 juillet :

In [4]:
enedis_final=enedis_final.loc[enedis_final["date"]<"2021-07-31"]

On ne conserve que la plage de puissance 3 à 6 kvA.   
En effet, c'est la plage de puissance qui est la plus présente :

In [5]:
enedis_final.groupby(["plage_de_puissance_souscrite"])["nb_points_soutirage"].sum().reset_index()

,plage_de_puissance_souscrite,nb_points_soutirage
0,P0: Total <= 36 kVA,327120123782
1,P1: ]0-3] kVA,15297211995
2,P1: ]0-6] kVA,62460154025
3,P1: ]0-9] kVA,1319614428
4,P2: ]3-6] kVA,129867441852
5,P3: ]6-9] kVA,75962610871
6,P4: ]9-12] kVA,28865683345
7,P5: ]12-15] kVA,4203540697
8,P6: ]15-18] kVA,7330414029
9,P6: ]15-36] kVA,35481533


Donc on conserve cette plage de puissance :

In [6]:
enedis_filtered=enedis_final[(enedis_final["plage_de_puissance_souscrite"]=="P2: ]3-6] kVA")]

In [7]:
enedis_filtered

,horodate,region_x,code_region,total_energie_soutiree_wh,plage_de_puissance_souscrite,nb_points_soutirage,jour_max_du_mois_0_1,semaine_max_du_mois_0_1,profil,profil_type,...,"Part des ""cadres et professions intellectuelles supérieures"" dans la population 2018",Médiane du revenu disponible par UC 2018,Population au dernier recensement 2018,Part des 15-29 ans 2018,Part des 30-44 ans 2018,Part des 45-59 ans 2018,Part des 60-74 ans 2018,Part des 75 ans et plus 2018,Taux de chômage des 15 ans et plus (RP) 2018,tmoy
0,2020-06-04T00:00:00+00:00,Auvergne-Rhône-Alpes,84,130666817.0,P2: ]3-6] kVA,1685439,0,0,RES1 (+ RES1WE),RES,...,9.4,22480.0,7994459.0,17.6,18.8,19.8,16.2,9.4,11.4,16.48
8,2020-06-04T00:00:00+00:00,Bourgogne-Franche-Comté,27,44987629.0,P2: ]3-6] kVA,550076,0,0,RES1 (+ RES1WE),RES,...,6.3,21480.0,2807807.0,16.2,17.3,20.1,18.6,10.9,12.0,17.17
89,2020-06-04T00:00:00+00:00,Hauts-de-France,32,92853270.0,P2: ]3-6] kVA,1101239,0,0,RES1 (+ RES1WE),RES,...,7.0,20110.0,6004108.0,18.5,18.8,19.5,15.9,7.9,16.3,13.66
95,2020-06-04T00:00:00+00:00,Île-de-France,11,238127892.0,P2: ]3-6] kVA,3097303,0,0,RES1 (+ RES1WE),RES,...,18.2,23860.0,12213447.0,20.0,21.2,19.4,13.2,6.8,12.1,14.51
151,2020-06-04T00:00:00+00:00,Grand-Est,44,69687114.0,P2: ]3-6] kVA,815710,0,0,RES1 (+ RES1WE),RES,...,7.2,21610.0,5550389.0,17.5,18.3,20.6,17.1,9.3,13.3,15.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4574503,2021-07-30T23:30:00+00:00,Hauts-de-France,32,97406596.0,P2: ]3-6] kVA,1119748,0,0,RES1 (+ RES1WE),RES,...,7.0,20110.0,6004108.0,18.5,18.8,19.5,15.9,7.9,16.3,16.72
4574519,2021-07-30T23:30:00+00:00,Île-de-France,11,225060904.0,P2: ]3-6] kVA,3133979,0,0,RES1 (+ RES1WE),RES,...,18.2,23860.0,12213447.0,20.0,21.2,19.4,13.2,6.8,12.1,18.68
4574587,2021-07-30T23:30:00+00:00,Bourgogne-Franche-Comté,27,48132464.0,P2: ]3-6] kVA,555237,0,0,RES1 (+ RES1WE),RES,...,6.3,21480.0,2807807.0,16.2,17.3,20.1,18.6,10.9,12.0,19.16
4574596,2021-07-30T23:30:00+00:00,Provence-Alpes-Côte d'Azur,93,91928076.0,P2: ]3-6] kVA,1026967,0,0,RES1 (+ RES1WE),RES,...,8.3,21350.0,5052832.0,16.1,17.9,20.1,18.0,11.1,14.0,24.61


On importe la bdd :

In [11]:
enedis_filtered.to_csv("Données/enedis_filtered.csv",index=False)